In [10]:
#from https://github.com/lyeoni/pytorch-mnist-VAE/blob/master/pytorch-mnist-VAE.ipynb

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

0it [00:00, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw


0it [00:00, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw


0it [00:00, ?it/s]

Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw


0it [00:00, ?it/s]

Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw
Processing...
Done!


In [24]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, h_dim3, z_dim):
        super(VAE, self).__init__()
    
        self.encode = nn.Sequential(# 28 -> 14
            nn.Conv2d(x_dim, h_dim1, 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(h_dim1),
            nn.LeakyReLU(0.1),
            nn.Conv2d(h_dim1, h_dim2, 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(h_dim2),
            nn.LeakyReLU(0.1),
            nn.Linear(h_dim2, h_dim3),
            nn.BatchNorm1d(h_dim3),
            nn.LeakyReLU(0.1),
        )
        self.fc31 = nn.Sequential(
            nn.Linear(h_dim3, z_dim),
            nn.Sigmoid(),
        )
        self.fc32 = nn.Sequential(
            nn.Linear(h_dim3, z_dim),
            nn.Sigmoid(),
        )

        self.decode =  nn.Sequential(
            nn.Linear(z_dim, h_dim3),
            nn.BatchNorm1d(h_dim3),
            nn.ReLU(),
            nn.Linear(h_dim3, h_dim2),
            nn.BatchNorm1d(h_dim2),
            nn.ReLU(),
            nn.ConvTranspose2d(h_dim2, h_dim1, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(h_dim1),
            nn.ReLU(),
            nn.ConvTranspose2d(h_dim1, x_dim, 4, stride=2, padding=1, bias=False),
            nn.Tanh(),
        )



    def encoder(self, x):
        print(x.shape)
        h = self.encode(x)
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        return self.decode(z)
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var


In [25]:

# build model
vae = VAE(x_dim=784, h_dim1= 1024, h_dim2=7*7*128, h_dim3 = 64, z_dim=12)
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
vae.to(device)

VAE(
  (encode): Sequential(
    (0): Conv2d(784, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
    (3): Conv2d(1024, 6272, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(6272, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.1)
    (6): Linear(in_features=6272, out_features=64, bias=True)
    (7): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.1)
  )
  (fc31): Sequential(
    (0): Linear(in_features=64, out_features=12, bias=True)
    (1): Sigmoid()
  )
  (fc32): Sequential(
    (0): Linear(in_features=64, out_features=12, bias=True)
    (1): Sigmoid()
  )
  (decode): Sequential(
    (0): Linear(in_features=12, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, moment

In [26]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [27]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))
    torch.save('vae.dat', vae)

In [28]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.to(device)
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [29]:
train(10)

torch.Size([100, 784])


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [1024, 784, 3, 3], but got 2-dimensional input of size [100, 784] instead